In [1]:
import time

import pandas as pd
import numpy as np

import tomotopy as tp
import pprint

import collections
import gensim
from konlpy.tag import Okt
from konlpy.corpus import kolaw

import matplotlib.pyplot as plt
import matplotlib.colors as clr

import matplotlib.font_manager as fm

fm.get_fontconfig_fonts()
fontpath = 'C:/Windows/Fonts/malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=fontpath).get_name()
plt.rc('font', family=font_name)

%matplotlib inline

%config InlineBackend.figure_format = 'retina'


In [2]:
df = pd.read_csv("최종.csv")
df['Date'] = df['Date'].astype(str)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop('Date',axis = 1)

,LINK,Title,Content,Species,Year,Month,Day
0,http://news.khan.co.kr/kh_news/khan_art_view.h...,삼성전자 차 부품전장 사업 수장 교체,팀장에 이승욱 부사장 선임LG전자와 경쟁 치열해질 듯 삼성전자가 미래 먹거리인 자동...,경향,2020,12,31
1,http://news.khan.co.kr/kh_news/khan_art_view.h...,25 시민 자율과 자치의 상상력을 발휘폭주하는 과학기술 말머리를 돌려라,기술 민주주의 얼마 전까지도 우리 사회 속 기술 수용은 그리 대중의 관심사가 아니었...,경향,2020,12,31
2,http://news.khan.co.kr/kh_news/khan_art_view.h...,오늘의 인사 2021년 1월 1일,국무조정실국무총리비서실 국민생명지키기추진단 총괄기획팀장 양소영 뉴미디어총괄행정...,경향,2020,12,31
3,http://news.khan.co.kr/kh_news/khan_art_view.h...,신축년 알리는 제야의 종 타종 행사 VR로 생생하게,SKT 서울시 홈페이지에 준비별도 기기 없이 PC스마트폰 감상 코로나19 확산으로 ...,경향,2020,12,30
4,http://news.khan.co.kr/kh_news/khan_art_view.h...,건축자재 유통 플랫폼 엘리콘 2020년도 DMC 이노베이션캠프 창업경진대회 우수상 수상,머신러닝 기반 건축자재 유통 플랫폼 엘리콘이 2020년도 DMC이노베이션캠프 창업경...,경향,2020,12,30
...,...,...,...,...,...,...,...
14100,https://www.donga.com/news/article/all/2010012...,동부건설 방범로봇 아파트 3758채 공급,동부건설은 올해 7개 단지에서 총 3758채를 공급한다고 26일 밝혔다 동부건설...,동아,2010,1,27
14101,https://www.donga.com/news/article/all/2010012...,반도체보다 빨리 크는 가전로봇시장,한국과학기술연구원은 인간형 로봇 마루가 구운 토스트와 전자레인지 안의 컵을 사람에...,동아,2010,1,20
14102,https://www.donga.com/news/article/all/2010011...,법인화 이후 도쿄대는 달라지고 있다,일본 도쿄대를 중심으로 2004년 모든 일본 국립대학이 법인화됐다 5년이 지난...,동아,2010,1,15
14103,https://www.donga.com/news/article/all/2010010...,녹색두뇌여 녹색혁명을 일으켜라,에코 지능 대니얼 골먼 지음이수경 옮김 360쪽1만8000원웅진지식하우스추운 2...,동아,2010,1,9


In [3]:
df

,LINK,Title,Content,Date,Species,Year,Month,Day
0,http://news.khan.co.kr/kh_news/khan_art_view.h...,삼성전자 차 부품전장 사업 수장 교체,팀장에 이승욱 부사장 선임LG전자와 경쟁 치열해질 듯 삼성전자가 미래 먹거리인 자동...,2020-12-31,경향,2020,12,31
1,http://news.khan.co.kr/kh_news/khan_art_view.h...,25 시민 자율과 자치의 상상력을 발휘폭주하는 과학기술 말머리를 돌려라,기술 민주주의 얼마 전까지도 우리 사회 속 기술 수용은 그리 대중의 관심사가 아니었...,2020-12-31,경향,2020,12,31
2,http://news.khan.co.kr/kh_news/khan_art_view.h...,오늘의 인사 2021년 1월 1일,국무조정실국무총리비서실 국민생명지키기추진단 총괄기획팀장 양소영 뉴미디어총괄행정...,2020-12-31,경향,2020,12,31
3,http://news.khan.co.kr/kh_news/khan_art_view.h...,신축년 알리는 제야의 종 타종 행사 VR로 생생하게,SKT 서울시 홈페이지에 준비별도 기기 없이 PC스마트폰 감상 코로나19 확산으로 ...,2020-12-30,경향,2020,12,30
4,http://news.khan.co.kr/kh_news/khan_art_view.h...,건축자재 유통 플랫폼 엘리콘 2020년도 DMC 이노베이션캠프 창업경진대회 우수상 수상,머신러닝 기반 건축자재 유통 플랫폼 엘리콘이 2020년도 DMC이노베이션캠프 창업경...,2020-12-30,경향,2020,12,30
...,...,...,...,...,...,...,...,...
14100,https://www.donga.com/news/article/all/2010012...,동부건설 방범로봇 아파트 3758채 공급,동부건설은 올해 7개 단지에서 총 3758채를 공급한다고 26일 밝혔다 동부건설...,2010-01-27,동아,2010,1,27
14101,https://www.donga.com/news/article/all/2010012...,반도체보다 빨리 크는 가전로봇시장,한국과학기술연구원은 인간형 로봇 마루가 구운 토스트와 전자레인지 안의 컵을 사람에...,2010-01-20,동아,2010,1,20
14102,https://www.donga.com/news/article/all/2010011...,법인화 이후 도쿄대는 달라지고 있다,일본 도쿄대를 중심으로 2004년 모든 일본 국립대학이 법인화됐다 5년이 지난...,2010-01-15,동아,2010,1,15
14103,https://www.donga.com/news/article/all/2010010...,녹색두뇌여 녹색혁명을 일으켜라,에코 지능 대니얼 골먼 지음이수경 옮김 360쪽1만8000원웅진지식하우스추운 2...,2010-01-09,동아,2010,1,9


In [4]:
linklist = []
for i in df['LINK']:
    linklist.append(i)
    

In [5]:
len(linklist)

14105

In [6]:
linklist.sort()
중복 = []

for i in range(0,(len(linklist)-1)):
    if linklist[i] == linklist[i+1]:
        중복.append(i)
        
len(중복) #중복갯수 확인

194

In [7]:
for i in 중복: #실제 중복인지 링크로 들어가서 확인
    print(f'{linklist[i]}, {linklist[i+1]} 은 같은 링크' )

https://www.donga.com/news/article/all/20151105/74605223/1, https://www.donga.com/news/article/all/20151105/74605223/1 은 같은 링크
https://www.donga.com/news/article/all/20160219/76533011/1, https://www.donga.com/news/article/all/20160219/76533011/1 은 같은 링크
https://www.donga.com/news/article/all/20160311/76934928/1, https://www.donga.com/news/article/all/20160311/76934928/1 은 같은 링크
https://www.donga.com/news/article/all/20160314/76975078/1, https://www.donga.com/news/article/all/20160314/76975078/1 은 같은 링크
https://www.donga.com/news/article/all/20160314/76975450/1, https://www.donga.com/news/article/all/20160314/76975450/1 은 같은 링크
https://www.donga.com/news/article/all/20160314/76975499/1, https://www.donga.com/news/article/all/20160314/76975499/1 은 같은 링크
https://www.donga.com/news/article/all/20160314/76975660/1, https://www.donga.com/news/article/all/20160314/76975660/1 은 같은 링크
https://www.donga.com/news/article/all/20160315/76996230/1, https://www.donga.com/news/article/all/20160315/769

In [8]:
df = pd.read_csv("최종.csv")
df['Date'] = df['Date'].astype(str)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df.drop('Date',axis = 1)

,LINK,Title,Content,Species,Year,Month,Day
0,http://news.khan.co.kr/kh_news/khan_art_view.h...,삼성전자 차 부품전장 사업 수장 교체,팀장에 이승욱 부사장 선임LG전자와 경쟁 치열해질 듯 삼성전자가 미래 먹거리인 자동...,경향,2020,12,31
1,http://news.khan.co.kr/kh_news/khan_art_view.h...,25 시민 자율과 자치의 상상력을 발휘폭주하는 과학기술 말머리를 돌려라,기술 민주주의 얼마 전까지도 우리 사회 속 기술 수용은 그리 대중의 관심사가 아니었...,경향,2020,12,31
2,http://news.khan.co.kr/kh_news/khan_art_view.h...,오늘의 인사 2021년 1월 1일,국무조정실국무총리비서실 국민생명지키기추진단 총괄기획팀장 양소영 뉴미디어총괄행정...,경향,2020,12,31
3,http://news.khan.co.kr/kh_news/khan_art_view.h...,신축년 알리는 제야의 종 타종 행사 VR로 생생하게,SKT 서울시 홈페이지에 준비별도 기기 없이 PC스마트폰 감상 코로나19 확산으로 ...,경향,2020,12,30
4,http://news.khan.co.kr/kh_news/khan_art_view.h...,건축자재 유통 플랫폼 엘리콘 2020년도 DMC 이노베이션캠프 창업경진대회 우수상 수상,머신러닝 기반 건축자재 유통 플랫폼 엘리콘이 2020년도 DMC이노베이션캠프 창업경...,경향,2020,12,30
...,...,...,...,...,...,...,...
14100,https://www.donga.com/news/article/all/2010012...,동부건설 방범로봇 아파트 3758채 공급,동부건설은 올해 7개 단지에서 총 3758채를 공급한다고 26일 밝혔다 동부건설...,동아,2010,1,27
14101,https://www.donga.com/news/article/all/2010012...,반도체보다 빨리 크는 가전로봇시장,한국과학기술연구원은 인간형 로봇 마루가 구운 토스트와 전자레인지 안의 컵을 사람에...,동아,2010,1,20
14102,https://www.donga.com/news/article/all/2010011...,법인화 이후 도쿄대는 달라지고 있다,일본 도쿄대를 중심으로 2004년 모든 일본 국립대학이 법인화됐다 5년이 지난...,동아,2010,1,15
14103,https://www.donga.com/news/article/all/2010010...,녹색두뇌여 녹색혁명을 일으켜라,에코 지능 대니얼 골먼 지음이수경 옮김 360쪽1만8000원웅진지식하우스추운 2...,동아,2010,1,9


In [9]:
df = df.drop_duplicates() #중복제거함수

In [10]:
linklist = []

for i in df['LINK']:
    linklist.append(i)
    
linklist.sort()
중복 = []

for i in range(0,(len(linklist)-1)):
    if linklist[i] == linklist[i+1]:
        중복.append(i)

print(len(중복)) #중복 0개 확인

0


In [11]:
df.to_csv('중복제거(진짜최종).csv', encoding = 'utf-8-sig', index_label = False, index = False)